In [2]:
import requests
from emo_utils import *
import emoji


In [3]:
import subprocess
NODE_PORT = subprocess.run("kubectl get services/emojifier -o go-template='{{(index .spec.ports 0).nodePort}}'",shell=True,stdout=subprocess.PIPE)
URL="http://localhost:%d/predict"%(int(NODE_PORT.stdout))
print(URL)

http://localhost:31483/predict


In [4]:
!kubectl get deployments

NAME        READY   UP-TO-DATE   AVAILABLE   AGE
emojifier   1/1     1            1           20h


In [5]:
#using multiprocessing so that we can make the http requests in parallel
#
from multiprocessing import Pool
import time






In [6]:
def predict_emoji(texts) :
    for text in texts:
        params = {'text' : text}
        r = requests.get(url= URL, params = params)
        #print(text,label_to_emoji(r.json()))

In [7]:
X_test, _ = read_csv('data/train_emoji.csv')
REPEAT_FACTOR = 40
# create a big dataset by repeating X_test REPEAT_FACTOR times

texts_nested = [X_test]*REPEAT_FACTOR
#flatten nested array
texts = np.array([x for l in texts_nested for x in l])


In [8]:
#need to have enough processes to overwhelm a single pod
NUM_PROCESSES = 8
p = Pool(NUM_PROCESSES)

def timed_sequence(p) :
    start_time = time.time()
    p.map(predict_emoji, np.split(texts,NUM_PROCESSES))
    end_time = time.time()
    print(end_time-start_time)


In [9]:
timed_sequence(p)

44.567094564437866


In [10]:
!kubectl scale deployments/emojifier --replicas=2
#Note : wait about 3 minutes for the python models to load models
# and be ready to process http requests
# We can do top on the worker nodes to see when python activity subsides

deployment.extensions/emojifier scaled


In [11]:
!kubectl get deployments

NAME        READY   UP-TO-DATE   AVAILABLE   AGE
emojifier   2/2     2            2           20h


In [12]:
timed_sequence(p)

31.08963179588318


In [13]:
!kubectl scale deployments/emojifier --replicas=8
#Note : wait about 3 minutes for the python models to load models
# and be ready to process http requests
# We can do top on the worker nodes to see when python activity subsides

deployment.extensions/emojifier scaled


In [14]:
!kubectl get deployments

NAME        READY   UP-TO-DATE   AVAILABLE   AGE
emojifier   8/8     8            8           20h


In [15]:
timed_sequence(p)

29.249848127365112


In [1]:
!kubectl scale deployments/emojifier --replicas=1

deployment.extensions/emojifier scaled
